In [5]:
import os
import pandas as pd
import numpy as np
import json
import ast

In [9]:
temp = "[0, 1, 2]"
temp = ast.literal_eval(temp)
type(temp)

list

In [38]:
results_path = 'models/ddpm/hypers/0.0025'
folders = sorted(os.listdir(results_path))

results = pd.DataFrame(columns=['IMG_SIZE',
 'BATCH_SIZE',
 'LEARNING_RATE',
 'NUM_EPOCHS',
 'NUM_GENERATE_IMAGES',
 'NUM_TIMESTEPS',
 'MIXED_PRECISION',
 'GRADIENT_ACCUMULATION_STEPS',
 'losses',
 'fid_scores',
 'fid_scores_epochs',
 'dataset',
 'UNET_size',
 'optimizer',
 'seed',
 'exited_loss_na',
 'execution_time'])

# get metadata.json from each folder
for folder in folders:
    metadata_path = os.path.join(results_path, folder, 'metadata.json')
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    metadata['losses'] = str(metadata['losses'])
    metadata['fid_scores'] = str(metadata['fid_scores'])
    metadata['fid_scores_epochs'] = str(metadata['fid_scores_epochs'])

    results = pd.concat([results, pd.DataFrame(metadata, index=[0])], ignore_index=True)
results.head()

/var/folders/nf/1p7pqpyx44q1v5y4d46cq8m80000gn/T/ipykernel_96274/255260940.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, pd.DataFrame(metadata, index=[0])], ignore_index=True)


,IMG_SIZE,BATCH_SIZE,LEARNING_RATE,NUM_EPOCHS,NUM_GENERATE_IMAGES,NUM_TIMESTEPS,MIXED_PRECISION,GRADIENT_ACCUMULATION_STEPS,losses,fid_scores,fid_scores_epochs,dataset,UNET_size,optimizer,seed,exited_loss_na,execution_time
0,96,16,0.0001,35,9,500,fp16,1,"[1.1774862110614777, 1.0176522086064022, 0.765...","[321.73444818864544, 300.4899763711119, 275.96...","[0, 10, 20, 30, 34]",square96_random0.0025,big,Adam,0,False,2233.623996
1,96,16,0.0001,35,9,500,fp16,1,"[1.1789524853229523, 1.017277967184782, 0.7649...","[326.84674433309567, 290.5935336576049, 256.09...","[0, 10, 20, 30, 34]",square96_random0.0025,big,Adam,1,False,2240.804714
2,96,16,0.0001,35,9,500,fp16,1,"[1.1785640964905422, 1.018510840833187, 0.7674...","[325.661124946291, 293.20694522449094, 252.497...","[0, 10, 20, 30, 34]",square96_random0.0025,big,Adam,2,False,2240.856870
3,96,16,0.0001,35,9,500,fp16,1,"[1.178771900633971, 1.0184324271976948, 0.7668...","[324.7122726505821, 286.7539453833973, 250.920...","[0, 10, 20, 30, 34]",square96_random0.0025,big,Adam,3,False,2241.236400
4,96,16,0.0001,35,9,500,fp16,1,"[1.1780003954966862, 1.0174272085229557, 0.766...","[326.31338600251524, 284.38310313636873, 249.8...","[0, 10, 20, 30, 34]",square96_random0.0025,big,Adam,4,False,2247.025025


In [39]:
results['last_fid'] = results['fid_scores'].apply(lambda x: ast.literal_eval(x)[-1])

In [42]:
results['LEARNING_RATE'].value_counts()

LEARNING_RATE
0.0001    90
Name: count, dtype: int64

In [43]:
results.groupby(['LEARNING_RATE', 'optimizer', 'UNET_size']).agg({'last_fid': 'mean', }).sort_values(by='last_fid', ascending=True)

last_fid
LEARNING_RATE optimizer UNET_size            
0.0001        Adam      small      249.126123
                        big        250.474787
                        mid        256.743597
              SGD       big        320.612453
                        small      327.949616
                        mid        337.284856